# MobileNet Block

In [1]:
import tensorflow as tf
from tensorflow.keras import layers



def bottleneck_block(x, expand=96, squeeze=16):
    m = layers.Conv2D(expand, (1,1), kernel_regularizer=tf.keras.regularizers.l2(0.0001), use_bias = False)(x)
    m = layers.BatchNormalization()(m)
    m = layers.Activation('relu')(m)
    m = layers.DepthwiseConv2D((3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0001), use_bias = False)(m)
    m = layers.BatchNormalization()(m)
    m = layers.Activation('relu')(m)
    m = layers.Conv2D(squeeze, (1,1), kernel_regularizer=tf.keras.regularizers.l2(0.0001), use_bias = False)(m)
    m = layers.BatchNormalization()(m)

    return layers.Add()([m, x])


input = layers.Input(shape=(32, 32, 3))
x = layers.Conv2D(16, 1, activation='relu', padding='same')(input)
for i in range (5):
    x = bottleneck_block(x)
flatten = layers.Flatten()(x)
dense = layers.Dense(10, activation="softmax")(flatten)
model = tf.keras.models.Model(inputs=input, outputs=dense)


(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
 
model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-3),
              loss='categorical_crossentropy',
              metrics=['acc'])
model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          validation_split=0.2)

print(model.evaluate(x_test,y_test))

# ShuffleNet Architecture